


Cross Validation & Naive Bayes Lab - SMS Spam Classification
===============
* orignally developed by Ankit Jain
* modified by Justin Breucop
* modified by Dylan Hercher

Data source: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

# Section 1)

## Cross Validation from Scratch

Let's build it the function together! The steps to cross validation are:
1. Randomly separate your training set into _k_ groups
2. For each group _k_:
>1. Train your model on the other groups
>2. Score your model using group _k_ as validation
>3. Save your score and move to your next group

3. Add your _k_ scores and divide by _k_ to get your average score

In [46]:
# Importing Packages 
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier

In [47]:
model = KNeighborsClassifier(10)

df = pd.DataFrame.from_csv('../data/titanic-train.csv',index_col=None)[['Age','Pclass','SibSp','Survived']].dropna()
to_predict = "Survived"
features=['Age','Pclass','SibSp']
data = df[features]
label = df[to_predict]
folds=5 #break data into 5 sets

In [48]:
data

,Age,Pclass,SibSp
0,22,3,1
1,38,1,1
2,26,3,0
3,35,1,1
4,35,3,0
6,54,1,0
7,2,3,3
8,27,3,0
9,14,2,1
10,4,3,1


In [49]:
label

0      0
1      1
2      1
3      1
4      0
6      0
7      0
8      1
9      1
10     1
11     1
12     0
13     0
14     0
15     1
16     0
18     0
20     0
21     1
22     1
23     1
24     0
25     1
27     0
30     0
33     0
34     0
35     0
37     0
38     0
      ..
856    1
857    1
858    1
860    0
861    0
862    1
864    0
865    1
866    1
867    0
869    1
870    0
871    1
872    0
873    0
874    1
875    1
876    0
877    0
879    1
880    1
881    0
882    0
883    0
884    0
885    0
886    0
887    1
889    1
890    0
Name: Survived, dtype: int64

Let's build in class!

In [69]:
#Function here:
from sklearn.cross_validation import train_test_split
def cross_validate_df(data,label,model,k):
    X = data #three different subsets of data..which is x?
    y = label
    score = 0
    for i in range(k):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1.0/k) 
        myknn = KNeighborsClassifier(3).fit(X_train,y_train)
        score += myknn.score(X_test, y_test)
        print myknn.score(X_test, y_test)
         
    return score/k

In [65]:
def cross_validate_df2(data,label,model,k):
    train_slices = []
    test_slices = []
    positions = data.index.values

    np.random.shuffle(positions)

    for i in range(k):
        section_start = i*len(data)/k
        section_end = (i+1)*len(data)/k
        test_slices.append(positions[section_start:section_end])

        train_p1 = positions[ :section_start]
        train_p2 = positions[ section_end:]
        train_combo = np.concatenate([train_p1, train_p2])
        train_slices.append(train_combo)
    scores = 0
    for train_s1, test_s1 in zip(train_slices, test_slices): #combines 2 lists in to one list of tuples
        model.fit(data.loc[train_s1],label.loc[train_s1]) #within training and testing set, give me matching values 
        k_score = model.score(data.loc[test_s1],label.loc[test_s1])
        scores += k_score
        print k_score
    return scores/k



In [70]:
cross_validate_df(data,label,model,5)

0.65034965035
0.65034965035
0.685314685315
0.629370629371
0.643356643357


0.65174825174825168

In [66]:
# Once Complete test your function here
cross_validate_df2(data,label,model,5)

0.697183098592
0.727272727273
0.615384615385
0.671328671329
0.643356643357


0.67090515118684135

### Built-in Cross Validation
There is also a very simple cross validation function provided by sklearn

In [71]:
from sklearn.cross_validation import cross_val_score

# Results of using the built-in cross validation
# Note: Default is Kfold, but within sklearn.cross_validation
# there are many types of validation that can be used
cross_val_score(model, data, label, cv=3)

array([ 0.64016736,  0.70168067,  0.69198312])

In [72]:
cross_val_score(model, data, label, cv=3)

array([ 0.64016736,  0.70168067,  0.69198312])

In [73]:
np.mean(cross_val_score(model, data, label, cv=3))

0.67794371954950439

# Section 2)
## Naive Bayes and SMS Spam Classification

In [74]:
import numpy as np
import pandas as pd

from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics


In [75]:
## READING IN THE DATA
df = pd.DataFrame.from_csv("../data/SMSSpamCollection.tsv",sep='\t',header=0,index_col=None)

In [76]:
# examine the data
df.head()

,label,msg
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [77]:
df[df.label=='spam'].head()

,label,msg
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."


In [78]:
df.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [79]:
df.msg.describe()

count                       5572
unique                      5169
top       Sorry, I'll call later
freq                          30
Name: msg, dtype: object

In [80]:
# Convert the label into a binary variable
# Remember the map function we learned before?
df['label'] = df.label.map({'ham': 0 , 'spam':1})

In [81]:
df.head()

,label,msg
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [82]:
# split into training and testing sets by calling sklearn lib
# by default, the data set is split into 0.75 (training) and 0.25 (testing)
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.msg, df.label, random_state=1)

In [83]:
print X_train.shape
print X_train

(4179,)
710     4mths half price Orange line rental & latest c...
3740                           Did you stitch his trouser
2711    Hope you enjoyed your new content. text stop t...
3155    Not heard from U4 a while. Call 4 rude chat pr...
3748    Ü neva tell me how i noe... I'm not at home in...
2389    wiskey Brandy Rum Gin Beer Vodka Scotch Shampa...
3464    i am seeking a lady in the street and a freak ...
772     Lol! U drunkard! Just doing my hair at d momen...
3667    I'm turning off my phone. My moms telling ever...
4955    U coming back 4 dinner rite? Dad ask me so i r...
854     AH POOR BABY!HOPE URFEELING BETTERSN LUV! PROB...
4079                  Gam gone after outstanding innings.
2837                         Nice.nice.how is it working?
1392                  Haha just kidding, papa needs drugs
5533    Hey chief, can you give me a bell when you get...
874     Ugh its been a long day. I'm exhausted. Just w...
4408    Awesome, plan to get here any time after like ...
3990  

In [84]:
X_test.shape #25 75 split

(1393,)

Now we need to convert the text into feature vectors which can be used for machine learning purposes.
We will use the scikit function of CountVectorizer to 'convert text into a matrix of token counts'

 http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

#### Lets try a simple example

In [85]:
from sklearn.feature_extraction.text import CountVectorizer

In [86]:
# start with a simple example
train_simple = ['call you tonight',
                'Call me a cab',
                'please call me... PLEASE!']

In [87]:
# learn the 'vocabulary' of the training data
vect = CountVectorizer(decode_error='ignore') #go thru text and get counts for train_simple
vect.fit(train_simple)
vect.get_feature_names()

[u'cab', u'call', u'me', u'please', u'tonight', u'you']

In [90]:
# transform training data into a 'document-term matrix'
train_simple_dtm = vect.transform(train_simple)
train_simple_dtm.toarray()

array([[0, 1, 0, 0, 1, 1],
       [1, 1, 1, 0, 0, 0],
       [0, 1, 1, 2, 0, 0]])

In [91]:
# We can see how we've adjusted our data easily!
# examine the vocabulary and document-term matrix together
print train_simple
    
pd.DataFrame(train_simple_dtm.toarray(), columns=vect.get_feature_names())

['call you tonight', 'Call me a cab', 'please call me... PLEASE!']


,cab,call,me,please,tonight,you
0,0,1,0,0,1,1
1,1,1,1,0,0,0
2,0,1,1,2,0,0


In [94]:
# transform testing data into a document-term matrix (using existing vocabulary)
test_simple = ["please don't call me"]
test_simple_dtm = vect.transform(test_simple)
test_simple_dtm.toarray()

pd.DataFrame(test_simple_dtm.toarray(), columns=vect.get_feature_names())

,cab,call,me,please,tonight,you
0,0,1,1,1,0,0


#### Question:  How does the above test_simple show how things can go wrong?

#### Exercise: Using the dataset below
   * Vectorize the text
   * Store the results in a DataFrame
   * Show word counts (hint: one dataframe describer can do this)
   * Transform the test text

In [100]:
train_exp = ['where is my taco?',
                'did I eat the taco',
                'I can easily eat my way through that whole box of tacos!',
                'I think way too much about tacos, huh',
                'taco, taco, taco!!!'                
               ]
test_exp = [
    'where did he go?', 'how long did the whole thing last', 'lets go eat one taco or multiple tacos'
]

In [111]:
from sklearn.feature_extraction.text import CountVectorizer

vect1 = CountVectorizer(decode_error='ignore')
vect1.fit(train_exp)
vect1.get_feature_names()
train_exp_dtm = vect.transform(train_exp)
train_exp_df = pd.DataFrame(train_exp_dtm.toarray(), columns= vect1.get_feature_names() )
train_exp_df.head()


ValueError: Shape of passed values is (6, 5), indices imply (21, 5)

## Vectorizing our SMS Dataset

In [112]:
# instantiate the vectorizer ( use variable name as vect)
vect = CountVectorizer(decode_error = 'ignore')
vect.fit(X_train)
vect.get_feature_names()

[u'00',
 u'000',
 u'008704050406',
 u'0121',
 u'01223585236',
 u'01223585334',
 u'0125698789',
 u'02',
 u'0207',
 u'02072069400',
 u'02073162414',
 u'02085076972',
 u'021',
 u'03',
 u'04',
 u'0430',
 u'05',
 u'050703',
 u'0578',
 u'06',
 u'07',
 u'07008009200',
 u'07090201529',
 u'07090298926',
 u'07123456789',
 u'07732584351',
 u'07734396839',
 u'07742676969',
 u'0776xxxxxxx',
 u'07781482378',
 u'07786200117',
 u'078',
 u'07801543489',
 u'07808',
 u'07808247860',
 u'07808726822',
 u'07815296484',
 u'07821230901',
 u'07880867867',
 u'0789xxxxxxx',
 u'07946746291',
 u'0796xxxxxx',
 u'07973788240',
 u'07xxxxxxxxx',
 u'08',
 u'0800',
 u'08000407165',
 u'08000776320',
 u'08000839402',
 u'08000930705',
 u'08000938767',
 u'08001950382',
 u'08002888812',
 u'08002986030',
 u'08002986906',
 u'08002988890',
 u'08006344447',
 u'0808',
 u'08081263000',
 u'08081560665',
 u'0825',
 u'083',
 u'0844',
 u'08448714184',
 u'0845',
 u'08450542832',
 u'08452810071',
 u'08452810073',
 u'08452810075over18',


In [113]:
X_train


710     4mths half price Orange line rental & latest c...
3740                           Did you stitch his trouser
2711    Hope you enjoyed your new content. text stop t...
3155    Not heard from U4 a while. Call 4 rude chat pr...
3748    Ü neva tell me how i noe... I'm not at home in...
2389    wiskey Brandy Rum Gin Beer Vodka Scotch Shampa...
3464    i am seeking a lady in the street and a freak ...
772     Lol! U drunkard! Just doing my hair at d momen...
3667    I'm turning off my phone. My moms telling ever...
4955    U coming back 4 dinner rite? Dad ask me so i r...
854     AH POOR BABY!HOPE URFEELING BETTERSN LUV! PROB...
4079                  Gam gone after outstanding innings.
2837                         Nice.nice.how is it working?
1392                  Haha just kidding, papa needs drugs
5533    Hey chief, can you give me a bell when you get...
874     Ugh its been a long day. I'm exhausted. Just w...
4408    Awesome, plan to get here any time after like ...
3990    Ok lor

In [114]:
# transform testing data into a document-term matrix: Use Variable name as test_dtm
train_dtm = vect.transform(X_train)
test_dtm = vect.transform(X_test)
print test_dtm

  (0, 1538)	1
  (0, 5189)	1
  (0, 6542)	1
  (0, 7405)	1
  (1, 1016)	1
  (1, 3050)	1
  (1, 4163)	1
  (1, 4238)	1
  (1, 4370)	1
  (1, 5200)	1
  (1, 6656)	1
  (1, 7407)	1
  (1, 7420)	1
  (2, 986)	1
  (2, 3244)	1
  (2, 7162)	1
  (3, 3237)	1
  (4, 887)	2
  (4, 1060)	1
  (4, 1595)	1
  (4, 2066)	1
  (4, 2833)	1
  (4, 3388)	1
  (4, 3623)	1
  (4, 3921)	1
  :	:
  (1391, 4373)	1
  (1391, 4413)	1
  (1391, 4441)	1
  (1391, 4743)	1
  (1391, 4778)	1
  (1391, 6017)	1
  (1391, 6057)	1
  (1391, 6829)	1
  (1391, 6904)	1
  (1391, 7012)	1
  (1391, 7120)	1
  (1391, 7230)	2
  (1391, 7239)	1
  (1391, 7287)	1
  (1391, 7357)	1
  (1392, 848)	1
  (1392, 2400)	1
  (1392, 2873)	1
  (1392, 3158)	1
  (1392, 4238)	1
  (1392, 4255)	2
  (1392, 4487)	1
  (1392, 4802)	1
  (1392, 5565)	1
  (1392, 7075)	1


In [116]:
# Get the length and names of the feature names
train_features = vect.get_feature_names()
len(train_features)

7456

In [117]:
train_features[:50]

[u'00',
 u'000',
 u'008704050406',
 u'0121',
 u'01223585236',
 u'01223585334',
 u'0125698789',
 u'02',
 u'0207',
 u'02072069400',
 u'02073162414',
 u'02085076972',
 u'021',
 u'03',
 u'04',
 u'0430',
 u'05',
 u'050703',
 u'0578',
 u'06',
 u'07',
 u'07008009200',
 u'07090201529',
 u'07090298926',
 u'07123456789',
 u'07732584351',
 u'07734396839',
 u'07742676969',
 u'0776xxxxxxx',
 u'07781482378',
 u'07786200117',
 u'078',
 u'07801543489',
 u'07808',
 u'07808247860',
 u'07808726822',
 u'07815296484',
 u'07821230901',
 u'07880867867',
 u'0789xxxxxxx',
 u'07946746291',
 u'0796xxxxxx',
 u'07973788240',
 u'07xxxxxxxxx',
 u'08',
 u'0800',
 u'08000407165',
 u'08000776320',
 u'08000839402',
 u'08000930705']

In [118]:
train_features[-50:]

[u'yer',
 u'yes',
 u'yest',
 u'yesterday',
 u'yet',
 u'yetunde',
 u'yijue',
 u'ym',
 u'ymca',
 u'yo',
 u'yoga',
 u'yogasana',
 u'yor',
 u'yorge',
 u'you',
 u'youdoing',
 u'youi',
 u'youphone',
 u'your',
 u'youre',
 u'yourjob',
 u'yours',
 u'yourself',
 u'youwanna',
 u'yowifes',
 u'yoyyooo',
 u'yr',
 u'yrs',
 u'ything',
 u'yummmm',
 u'yummy',
 u'yun',
 u'yunny',
 u'yuo',
 u'yuou',
 u'yup',
 u'zac',
 u'zaher',
 u'zealand',
 u'zebra',
 u'zed',
 u'zeros',
 u'zhong',
 u'zindgi',
 u'zoe',
 u'zoom',
 u'zouk',
 u'zyada',
 u'\xe8n',
 u'\u3028ud']

In [119]:
# convert train_dtm to a regular array
train_arr = train_dtm.toarray()
train_arr

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [120]:

# Revisit Numpy
arr = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])
print arr[0, 0]
print arr[1, 3]
print arr[0, :]
print arr[:, 0]
print np.sum(arr)
print np.sum(arr,axis = 0)
print np.sum(arr,axis = 1)




1
8
[1 2 3 4]
[1 5]
36
[ 6  8 10 12]
[10 26]


In [125]:
# exercise: calculate the number of tokens in the 0th message in train_arr
print np.sum(train_arr[0,:])


24


array([0, 0, 0, ..., 0, 0, 0])

In [123]:
train_arr[0,:]

array([0, 0, 0, ..., 0, 0, 0])

In [124]:

# exercise: count how many times the 0th token appears across ALL messages in train_arr
print np.sum(train_arr[:,0])

5


In [126]:
# exercise: count how many times EACH token appears across ALL messages in train_arr
print np.sum(train_arr, axis=0)

[ 5 23  2 ...,  1  1  1]


In [168]:
tolkendf = pd.DataFrame(np.sum(train_arr, axis=0))
tolkendf.columns = ['count']
tolkendf

,count
0,5
1,23
2,2
3,1
4,1
5,2
6,1
7,4
8,3
9,1


In [172]:
# exercise: create a DataFrame of tokens with their counts.

tolkendfFeat = pd.DataFrame(train_features)
tolkendfFeat.columns = ['tolken']


In [173]:
tolkenFinal = tolkendf.join(tolkendfFeat)
tolkenFinal

,count,tolken
0,5,00
1,23,000
2,2,008704050406
3,1,0121
4,1,01223585236
5,2,01223585334
6,1,0125698789
7,4,02
8,3,0207
9,1,02072069400


### Let's build the model with Naive Bayes Now

http://scikit-learn.org/stable/modules/naive_bayes.html

In [143]:
# train a Naive Bayes model using train_dtm
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(train_dtm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [144]:
# make predictions on test data using test_dtm
preds = nb.predict(test_dtm)
preds

array([0, 0, 0, ..., 0, 1, 0])

In [145]:
# compare predictions to true labels
from sklearn import metrics
print metrics.accuracy_score(y_test, preds)
print metrics.confusion_matrix(y_test, preds)

# confusion matrix: http://en.wikipedia.org/wiki/Confusion_matrix -- matrix row 1: prediction - misses  

0.988513998564
[[1203    5]
 [  11  174]]


In [146]:
# exercise: show the message text for the false positives
X_test[(y_test == 0) & (preds == 1)]

574               Waiting for your call.
3375             Also andros ice etc etc
45      No calls..messages..missed calls
3415             No pic. Please re-send.
1988    No calls..messages..missed calls
Name: msg, dtype: object

In [147]:
# exercise: show the message text for the false negatives
X_test[y_test > preds]
# or
X_test[(y_test == 1) & (preds == 0)]

3132    LookAtMe!: Thanks for your purchase of a video...
5       FreeMsg Hey there darling it's been 3 week's n...
3530    Xmas & New Years Eve tickets are now on sale f...
684     Hi I'm sue. I am 20 years old and work as a la...
1875    Would you like to see my XXX pics they are so ...
1893    CALL 09090900040 & LISTEN TO EXTREME DIRTY LIV...
4298    thesmszone.com lets you send free anonymous an...
4949    Hi this is Amy, we will be sending you a free ...
2821    INTERFLORA - It's not too late to order Inter...
2247    Hi ya babe x u 4goten bout me?' scammers getti...
4514    Money i have won wining number 946 wot do i do...
Name: msg, dtype: object

In [ ]:
## USING ALL DATA AND CROSS-VALIDATION, run NB again


In [150]:
# Here is the condensed code needed for reference
df = pd.DataFrame.from_csv("../data/SMSSpamCollection.tsv",sep='\t',header=0,index_col=None)

df.label = df.label.map({'ham':0, 'spam':1})
X_train, X_test, y_train, y_test = train_test_split(df.msg, df.label, random_state=1)

vect = CountVectorizer(decode_error = 'ignore')

vect.fit(X_train)
vect.get_feature_names()

train_dtm = vect.transform(X_train)
test_dtm = vect.transform(X_test)

In [166]:
## EXERCISE: CALCULATE THE 'SPAMMINESS' OF EACH TOKEN

vect.get_feature_names()
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(train_dtm, y_train)

preds = nb.predict(test_dtm)
from sklearn import metrics
print metrics.accuracy_score(y_test, preds)
print metrics.confusion_matrix(y_test, preds)


ham_df = pd.DataFrame(X_test[(y_test == 0)])
ham_df


# create separate DataFrames for ham and spam ( df_ham and df_spam)

0.988513998564
[[1203    5]
 [  11  174]]


,msg
1078,"Yep, by the pretty sculpture"
4028,"Yes, princess. Are you going to make me moan?"
958,Welp apparently he retired
4642,Havent.
4674,I forgot 2 ask ü all smth.. There's a card on ...
5461,Ok i thk i got it. Then u wan me 2 come now or...
4210,I want kfc its Tuesday. Only buy 2 meals ONLY ...
4216,No dear i was sleeping :-P
1603,Ok pa. Nothing problem:-)
1504,Ill be there on &lt;#&gt; ok.


In [178]:
preds = nb.predict(test_dtm)
preds

array([0, 0, 0, ..., 0, 1, 0])

In [179]:
vect.get_feature_names()

[u'00',
 u'000',
 u'000pes',
 u'008704050406',
 u'0089',
 u'0121',
 u'01223585236',
 u'01223585334',
 u'0125698789',
 u'02',
 u'0207',
 u'02072069400',
 u'02073162414',
 u'02085076972',
 u'021',
 u'03',
 u'04',
 u'0430',
 u'05',
 u'050703',
 u'0578',
 u'06',
 u'07',
 u'07008009200',
 u'07046744435',
 u'07090201529',
 u'07090298926',
 u'07099833605',
 u'07123456789',
 u'0721072',
 u'07732584351',
 u'07734396839',
 u'07742676969',
 u'07753741225',
 u'0776xxxxxxx',
 u'07781482378',
 u'07786200117',
 u'077xxx',
 u'078',
 u'07801543489',
 u'07808',
 u'07808247860',
 u'07808726822',
 u'07815296484',
 u'07821230901',
 u'078498',
 u'07880867867',
 u'0789xxxxxxx',
 u'07946746291',
 u'0796xxxxxx',
 u'07973788240',
 u'07xxxxxxxxx',
 u'08',
 u'0800',
 u'08000407165',
 u'08000776320',
 u'08000839402',
 u'08000930705',
 u'08000938767',
 u'08001950382',
 u'08002888812',
 u'08002986030',
 u'08002986906',
 u'08002988890',
 u'08006344447',
 u'0808',
 u'08081263000',
 u'08081560665',
 u'0825',
 u'083',
 

In [187]:
spam_df2preds = pd.DataFrame(preds)
spam_df2preds

,0
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [188]:
spam_df2feat = pd.DataFrame(vect.get_feature_names())
spam_df2feat
#spamFinal = spam_df2preds.join(spam_df2feat)

,0
0,00
1,000
2,000pes
3,008704050406
4,0089
5,0121
6,01223585236
7,01223585334
8,0125698789
9,02


In [167]:
spam_df = pd.DataFrame(X_test[(y_test == 1)])
spam_df



,msg
147,FreeMsg Why haven't you replied to my text? I'...
4517,Congrats! 2 mobile 3G Videophones R yours. cal...
3316,FREE MESSAGE Activate your 500 FREE Text Messa...
1929,Call from 08702490080 - tells u 2 call 0906635...
1745,Someone has conacted our dating service and en...
5189,ree entry in 2 a weekly comp for a chance to w...
1573,Ur cash-balance is currently 500 pounds - to m...
5567,This is the 2nd time we have tried 2 contact u...
2473,Final Chance! Claim ur £150 worth of discount ...
3807,URGENT! We are trying to contact you. Last wee...


In [174]:
df_ham=df[df.label==0]
df_spam=df[df.label==0]

In [177]:
vect.fit(df.msg)
all_features = vect.get_feature_names()

In [ ]:
# learn the vocabulary of ALL messages and save it


In [ ]:
# create document-term matrix of ham, then convert to a regular array


In [ ]:
# create document-term matrix of spam, then convert to a regular array


In [ ]:
# count how many times EACH token appears across ALL messages in ham_arr


In [ ]:
# count how many times EACH token appears across ALL messages in spam_arr


In [ ]:
# create a DataFrame of tokens with their separate ham and spam counts


In [ ]:
# add one to ham counts and spam counts so that ratio calculations (below) make more sensse


In [ ]:
# calculate ratio of spam-to-ham for each token


In [ ]:
# advanced: implement your own naive bayes classifier
